In [ ]:
!pip install catboost

In [ ]:
# prompt: auto EDA libraries in python like Sweetviz?
import pandas as pd

# standard libraries
import pandas as pd
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')

# ML/stats
from mlxtend.plotting import heatmap
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
# for splitting the data
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler



# visualizations
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns
import plotly.express as px
from tabulate import tabulate


%matplotlib inline

# tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#for text pre-pricessing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric



#bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#for word embedding
import gensim
from gensim.models import Word2Vec

#google collab drive read data
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/WIDS-Datathon-2024/Data/Train/'
train_df = pd.read_csv(path + 'training.csv')
print(train_df.head())

test_path = '/content/drive/MyDrive/WIDS-Datathon-2024/Data/Test/'
test_df = pd.read_csv(test_path + 'test.csv')

   patient_id patient_race  payer_type patient_state  patient_zip3  \
0      475714          NaN    MEDICAID            CA           924   
1      349367        White  COMMERCIAL            CA           928   
2      138632        White  COMMERCIAL            TX           760   
3      617843        White  COMMERCIAL            CA           926   
4      817482          NaN  COMMERCIAL            ID           836   

   patient_age patient_gender    bmi breast_cancer_diagnosis_code  \
0           84              F    NaN                       C50919   
1           62              F  28.49                       C50411   
2           43              F  38.09                       C50112   
3           45              F    NaN                       C50212   
4           55              F    NaN                         1749   

                        breast_cancer_diagnosis_desc  ...   disabled  \
0  Malignant neoplasm of unsp site of unspecified...  ...  12.871429   
1  Malig neoplm of u

In [ ]:
def sort_missing_percentages(dataframe, fields):
    """
    Calculate and sort the percentage of missing values for the specified fields in a DataFrame.

    Parameters:
    - dataframe: pandas DataFrame containing the data.
    - fields: List of strings representing the column names to check for missing values.

    Returns:
    - A pandas Series with the fields sorted by their percentage of missing values in descending order.
    """
    # Calculate the percentage of missing values for each field
    missing_percentages = dataframe[fields].isnull().mean() * 100

    # Sort the percentages in descending order
    missing_percentages_sorted = missing_percentages.sort_values(ascending=False)

    return missing_percentages_sorted

# Example usage:
# Assuming `train_df` is your DataFrame loaded with data
fields = [
    "patient_id", "breast_cancer_diagnosis_code", "patient_age",
    "metastatic_cancer_diagnosis_code", "payer_type", "patient_race",
    "patient_zip3", "patient_state", "income_household_25_to_35",
    "rent_burden", "income_household_under_5", "DiagPeriodL90D", "Division", "income_household_median", "income_household_under_5", "income_household_5_to_10", "income_household_10_to_15",
    "income_household_15_to_20", "income_household_20_to_25", "income_household_25_to_35", "income_household_35_to_50", "income_household_50_to_75", "income_household_75_to_100", "income_household_100_to_150", "income_household_150_over", "income_individual_median"
]


sorted_missing_percentages = sort_missing_percentages(train_df, fields)
print("sorted_missing_percentages", sorted_missing_percentages)

sorted_missing_percentages patient_race                        49.473113
payer_type                          13.970246
Division                             0.402913
patient_state                        0.395165
income_household_median              0.030993
income_household_50_to_75            0.030993
income_household_15_to_20            0.030993
income_household_10_to_15            0.030993
income_household_5_to_10             0.030993
income_household_under_5             0.030993
income_household_35_to_50            0.030993
rent_burden                          0.030993
income_household_under_5             0.030993
income_household_25_to_35            0.030993
income_household_25_to_35            0.030993
income_household_75_to_100           0.030993
income_household_100_to_150          0.030993
income_household_150_over            0.030993
income_household_20_to_25            0.030993
income_individual_median             0.007748
breast_cancer_diagnosis_code         0.000000
DiagPer

missing percentage

patient_race - 49.473113

payer_type - 13.970246

patient_state                        0.395165

income_household_25_to_35            0.030993

rent_burden                          0.030993

income_household_under_5             0.030993

patient_id                           0.000000

breast_cancer_diagnosis_code         0.000000

patient_age                          0.000000

metastatic_cancer_diagnosis_code     0.000000

patient_zip3                         0.000000

DiagPeriodL90D                       0.000000

In [ ]:
#shape of data
print(train_df.shape)
print(test_df.shape)
test_df.columns

(12906, 83)
(5792, 82)


Index(['patient_id', 'patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'patient_age', 'patient_gender', 'bmi',
       'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc',
       'metastatic_cancer_diagnosis_code', 'metastatic_first_novel_treatment',
       'metastatic_first_novel_treatment_type', 'Region', 'Division',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
       'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_50',
       'income_household_50_to_75', 'income_household_75_to_100',
       'income_household_100_to_150', 'in

In [ ]:
# prompt: drop bmi, metastatic_first_novel_treatment and metastatic_first_novel_treatment_type columns

train_df.drop(columns=['bmi', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type'], axis=1, inplace=True)
test_df.drop(columns=['bmi', 'metastatic_first_novel_treatment', 'metastatic_first_novel_treatment_type'], axis=1, inplace=True)


In [ ]:
split_path = '/content/drive/MyDrive/WIDS-Datathon-2024/split/'

train_df = pd.read_csv(split_path + 'train_set.csv')
val_df = pd.read_csv(split_path + 'val_set.csv')
print(train_df.shape)
print(val_df.shape)

(10324, 80)
(2582, 80)


In [ ]:
#get the datatypes of all columns
data_types = train_df.dtypes
#categorical_cols = data_types[data_types == 'object'].index.tolist()
categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()

# Identify numerical columns
#numerical_cols = data_types[data_types.isin(['int64', 'float64'])].index.tolist()
numerical_cols = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()

if "patient_zip3" in numerical_cols:
    numerical_cols.remove("patient_zip3")

# Add "patient_zip3" to category_cols if it's not already present
if "patient_zip3" not in categorical_cols:
    categorical_cols.append("patient_zip3")


print("Categorical columns:", categorical_cols)
print("Numerical columns:", numerical_cols)

Categorical columns: ['patient_race', 'payer_type', 'patient_state', 'patient_gender', 'breast_cancer_diagnosis_code', 'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'Region', 'Division', 'patient_zip3']
Numerical columns: ['patient_id', 'patient_age', 'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19', 'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s', 'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married', 'widowed', 'family_size', 'family_dual_income', 'income_household_median', 'income_household_under_5', 'income_household_5_to_10', 'income_household_10_to_15', 'income_household_15_to_20', 'income_household_20_to_25', 'income_household_25_to_35', 'income_household_35_to_50', 'income_household_50_to_75', 'income_household_75_to_100', 'income_household_100_to_150', 'income_household_150_over', 'income_household_six_figure', 'income_individual_median', 'home_ownership', 'housing_units', 'home_value', 'rent_median'

In [ ]:

def calculate_imputation_values(df, categorical_cols, numerical_cols, target_variable, patient_id):
    """
    Calculates imputation values (mean and mode) for numerical and categorical columns respectively.

    Parameters:
    df (pd.DataFrame): The DataFrame to calculate values from.
    categorical_cols (list): List of categorical column names.
    numerical_cols (list): List of numerical column names.
    target_variable (str): The name of the target variable column.
    patient_id (str): The patient identifier column.

    Returns:
    dict: A dictionary containing imputation values for each column.
    """
    imputation_values = {}
    for col in categorical_cols:
      imputation_values[col] = df[col].mode()[0]

    for col in numerical_cols:
        if col != target_variable and col != patient_id:
            imputation_values[col] = df[col].mean()

    return imputation_values


def apply_imputation(df, imputation_values):
    """
    Applies imputation values to a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame to impute.
    imputation_values (dict): Dictionary containing imputation values for each column.

    Returns:
    pd.DataFrame: The imputed DataFrame.
    """
    for col, value in imputation_values.items():
        df[col].fillna(value, inplace=True)

    return df


# Usage
# First, calculate imputation values from the training data
imputation_values = calculate_imputation_values(train_df, categorical_cols, numerical_cols, 'DiagPeriodL90D', 'patient_id')

# Apply imputation to training data
train_df_imputed = apply_imputation(train_df, imputation_values)

# Apply the same imputation to test data
test_df_imputed = apply_imputation(test_df, imputation_values)

# Apply the validation imputation to test data
val_df_imputed = apply_imputation(val_df, imputation_values)


In [ ]:
print(train_df_imputed.shape)
print(test_df_imputed.shape)
print(val_df_imputed.shape)



(12906, 83)
(5792, 82)
(2582, 80)


In [ ]:
missing = train_df_imputed.isna().sum()
missing

patient_id        0
patient_race      0
payer_type        0
patient_state     0
patient_zip3      0
                 ..
veteran           0
Ozone             0
PM25              0
N02               0
DiagPeriodL90D    0
Length: 83, dtype: int64

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

def perform_anova_for_categoricals(df, continuous_var, categorical_vars):
    anova_results = {}

    for cat_var in categorical_vars:
        # Prepare the formula for the ols model
        formula = f'{continuous_var} ~ C({cat_var})'

        # Fit the model
        model = ols(formula, data=df).fit()

        # Perform ANOVA and store the results
        anova_table = sm.stats.anova_lm(model, typ=2)
        anova_results[cat_var] = anova_table.loc['C('+cat_var+')', 'PR(>F)']

    return anova_results

# List of categorical variables in your dataset
categorical_vars = [
    "breast_cancer_diagnosis_code", "metastatic_cancer_diagnosis_code",
    "payer_type", "patient_race", "patient_zip3", "patient_state"

]

# Assuming 'train_df_imputed' is your DataFrame and 'patient_age' is the continuous variable
results = perform_anova_for_categoricals(train_df_imputed, 'patient_age', categorical_vars)

# Displaying the p-values from ANOVA tests for each categorical variable
for var, p_value in results.items():
    print(f"{var}: p-value = {p_value}")


breast_cancer_diagnosis_code: p-value = 5.658490329022965e-09
metastatic_cancer_diagnosis_code: p-value = 5.03476581963149e-33
payer_type: p-value = 0.0
patient_race: p-value = 1.7002061596806336e-22
patient_zip3: p-value = 1.0216520076740923e-40
patient_state: p-value = 1.0834236272342356e-62


In [ ]:
print(train_df_imputed['DiagPeriodL90D'].value_counts())
print(val_df_imputed['DiagPeriodL90D'].value_counts())

1    6448
0    3876
Name: DiagPeriodL90D, dtype: int64
1    1612
0     970
Name: DiagPeriodL90D, dtype: int64


In [ ]:
#get x_train and y_train
X_train = train_df_imputed.drop(['DiagPeriodL90D','patient_id'], axis=1)
y_train = train_df_imputed['DiagPeriodL90D']

In [ ]:
#check variables patient_zip3, DiagPeriodL90D and patient_id, if they are placed in their datatypes
print('"patient_zip3" in continuous_cols:', "patient_zip3" in numerical_cols)
print('"patient_zip3" in category_cols:', "patient_zip3" in categorical_cols)


print('"DiagPeriodL90D" in continuous_cols:', "DiagPeriodL90D" in numerical_cols)
print('"DiagPeriodL90D" in category_cols:', "DiagPeriodL90D" in categorical_cols)



print('"patient_id" in continuous_cols:', "patient_id" in numerical_cols)


"patient_zip3" in continuous_cols: False
"patient_zip3" in category_cols: True
"DiagPeriodL90D" in continuous_cols: True
"DiagPeriodL90D" in category_cols: False
"patient_id" in continuous_cols: True


In [ ]:
categorical_features_names = categorical_cols
categorical_features_indices = [X_train.columns.get_loc(col) for col in categorical_features_names]
categorical_features_names

['patient_race',
 'payer_type',
 'patient_state',
 'patient_gender',
 'breast_cancer_diagnosis_code',
 'breast_cancer_diagnosis_desc',
 'metastatic_cancer_diagnosis_code',
 'Region',
 'Division',
 'patient_zip3']

In [ ]:
from catboost import CatBoostClassifier, Pool

text_features = ['breast_cancer_diagnosis_desc']


categorical_features_indices = [X_train.columns.get_loc(col) for col in categorical_features_names if col not in text_features]


# Initialize CatBoostClassifier with text_features
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.01,
    depth=8,
    verbose=False,
    random_seed=1066,
    loss_function='Logloss',
    text_features=text_features
)



# Correctly create a Pool object with both cat_features and text_features
train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=categorical_features_indices,
    text_features=text_features
)

model.fit(train_pool, early_stopping_rounds=50)


feature_importances = model.get_feature_importance()

feature_names = X_train.columns
importance_dict = dict(zip(feature_names, feature_importances))

# Sort features by importance
sorted_features = sorted(importance_dict.items(), key=lambda item: item[1], reverse=True)

for name, importance in sorted_features:
    print(f"{name}: {importance}")


breast_cancer_diagnosis_code: 38.50595299302745
breast_cancer_diagnosis_desc: 6.3418781478294814
metastatic_cancer_diagnosis_code: 4.998319122434428
patient_age: 4.71944924987127
patient_zip3: 2.3492554567753565
payer_type: 2.1979562888846633
patient_race: 2.175816627619321
patient_state: 1.9056804122046362
Division: 1.472461499473944
Region: 1.0246836079194117
income_household_25_to_35: 0.8678326853017696
rent_burden: 0.8235840476245604
income_household_under_5: 0.8076816287289708
age_40s: 0.77058098636261
education_less_highschool: 0.754758686644044
unemployment_rate: 0.7280115211065475
PM25: 0.7182937467296954
self_employed: 0.6960847245745889
commute_time: 0.6931740234686359
male: 0.6930415301980378
education_some_college: 0.6773750932398249
race_native: 0.663198676743902
age_over_80: 0.6617906435663459
education_highschool: 0.6569719674183806
health_uninsured: 0.6377087798516485
divorced: 0.6332064673107894
age_20s: 0.6147348146971915
Ozone: 0.6072673960192014
widowed: 0.601832179

Grid search - Cross Validation to find the best parameters for Catboost to find the appropriate feature importance scores.

In [ ]:
'''
from catboost import CatBoostClassifier, Pool, cv

# Assuming X_train, y_train are defined, and categorical and text features are specified
train_pool = Pool(data=X_train, label=y_train, cat_features=categorical_features_indices, text_features=text_features)

# Define the parameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [6, 8, 10],
    'iterations': [100, 500, 1000]
}

# Initialize a CatBoostClassifier
model = CatBoostClassifier(
    verbose=False,
    random_seed=1066,
    loss_function='Logloss',
    text_features=text_features
)

# Perform grid search
grid_search_result = model.grid_search(param_grid, train_pool, cv=5, stratified=True, partition_random_seed=1066, verbose=False)

best_params = grid_search_result['params']
best_params
'''


bestTest = 0.4898056461
bestIteration = 99


bestTest = 0.4565613138
bestIteration = 99


bestTest = 0.4574808688
bestIteration = 47


bestTest = 0.4561696864
bestIteration = 499


bestTest = 0.4565014371
bestIteration = 99


bestTest = 0.4566946944
bestIteration = 44


bestTest = 0.4559669476
bestIteration = 724


bestTest = 0.4565014371
bestIteration = 99


bestTest = 0.4566946944
bestIteration = 44


bestTest = 0.4891330608
bestIteration = 99


bestTest = 0.4592922768
bestIteration = 61


bestTest = 0.459450233
bestIteration = 44


bestTest = 0.4560657864
bestIteration = 499


bestTest = 0.4571072414
bestIteration = 81


bestTest = 0.458971346
bestIteration = 38


bestTest = 0.4558829598
bestIteration = 586


bestTest = 0.4571072414
bestIteration = 81


bestTest = 0.458971346
bestIteration = 38


bestTest = 0.4891730131
bestIteration = 99


bestTest = 0.4614127865
bestIteration = 67


bestTest = 0.4613832754
bestIteration = 25


bestTest = 0.457218065
bestIteration = 382


bestTest

{'depth': 8, 'iterations': 1000, 'learning_rate': 0.01}

In [ ]:
import plotly.graph_objects as go

# Assuming 'feature_importances' and 'feature_names' are already obtained from the previous steps

# Filter out 'patient_zip3' and 'patient_id' before sorting
filtered_features = [(imp, name) for imp, name in zip(feature_importances, feature_names)]

# Sorting the filtered features by importance and selecting the top 10 (or top 13 as per your updated request)
sorted_features = sorted(filtered_features, key=lambda x: x[0], reverse=True)[:10]  # Change to 13 if you need top 13 instead of top 10

# Unzipping scores and names
scores, names = zip(*sorted_features)

# Creating the Plotly figure
fig = go.Figure(go.Bar(
    x=scores,
    y=names,
    orientation='h'
))

# Reversing the y-axis to have the feature with the highest importance at the top
fig.update_layout(yaxis=dict(autorange="reversed"),
                  title="Top Important Features Excluding patient_zip3 and patient_id",
                  xaxis_title="Feature Importance Score",
                  yaxis_title="Feature Names")

# Show the plot
fig.show()


# compute the percenatge of missing values for the below fields

In [ ]:
def sort_missing_percentages(dataframe, fields):
    """
    Calculate and sort the percentage of missing values for the specified fields in a DataFrame.

    Parameters:
    - dataframe: pandas DataFrame containing the data.
    - fields: List of strings representing the column names to check for missing values.

    Returns:
    - A pandas Series with the fields sorted by their percentage of missing values in descending order.
    """
    # Calculate the percentage of missing values for each field
    missing_percentages = dataframe[fields].isnull().mean() * 100

    # Sort the percentages in descending order
    missing_percentages_sorted = missing_percentages.sort_values(ascending=False)

    return missing_percentages_sorted

# Example usage:
# Assuming `train_df` is your DataFrame loaded with data
fields = [
    "patient_id", "breast_cancer_diagnosis_code", "patient_age",
    "metastatic_cancer_diagnosis_code", "payer_type", "patient_race",
    "patient_zip3", "patient_state", "income_household_25_to_35",
    "rent_burden", "income_household_under_5", "DiagPeriodL90D"
]


sorted_missing_percentages_trainset_df = sort_missing_percentages(train_df, fields)

sorted_missing_percentages_valset_df = sort_missing_percentages(val_df, fields)


# Print the sorted percentages of missing values
print("sorted_missing_percentages_trainset_df", sorted_missing_percentages_trainset_df, "\n")

print("sorted_missing_percentages_valset_df", sorted_missing_percentages_valset_df)



sorted_missing_percentages_trainset_df patient_race                        49.476947
payer_type                          14.025571
patient_state                        0.358388
income_household_25_to_35            0.038745
rent_burden                          0.038745
income_household_under_5             0.038745
patient_id                           0.000000
breast_cancer_diagnosis_code         0.000000
patient_age                          0.000000
metastatic_cancer_diagnosis_code     0.000000
patient_zip3                         0.000000
DiagPeriodL90D                       0.000000
dtype: float64 

sorted_missing_percentages_valset_df patient_race                        49.457785
payer_type                          13.749032
patient_state                        0.542215
patient_id                           0.000000
breast_cancer_diagnosis_code         0.000000
patient_age                          0.000000
metastatic_cancer_diagnosis_code     0.000000
patient_zip3                     

In [ ]:
train_data= train_df_imputed[['breast_cancer_diagnosis_code','breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code', 'patient_age', 'patient_zip3', 'payer_type', 'patient_race', 'patient_state', 'Division', 'Region', 'DiagPeriodL90D',
                                     'income_household_25_to_35', 'rent_burden', 'income_household_under_5', 'age_40s', 'education_less_highschool', 'unemployment_rate', 'PM25']]

There are no continuous variables other than patient age. Hence, there is no necessity to find pearson correlation

In [ ]:
import pandas as pd
import numpy as np

def find_highly_correlated_pairs(dataframe, threshold=0.7):
    """
    Finds and returns pairs of highly correlated features from the dataframe, along with their correlation scores.

    Parameters:
    - dataframe: A pandas DataFrame with the data.
    - threshold: A float representing the correlation coefficient threshold to consider.

    Returns:
    - A pandas DataFrame with each row containing a pair of highly correlated features and their correlation coefficient.
    """
    correlation_matrix = dataframe.corr(numeric_only=True)
    correlated_pairs = []

    print(correlated_pairs)


    # Use the np.triu (upper triangle) function to ignore lower half & diagonal entries
    for i, j in zip(*np.where(np.triu(np.abs(correlation_matrix) > threshold, k=1))):
        correlated_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j], correlation_matrix.iloc[i, j]))

    # Convert list of tuples into DataFrame for better readability
    correlated_pairs_df = pd.DataFrame(correlated_pairs, columns=['Feature 1', 'Feature 2', 'Correlation'])

    return correlated_pairs_df

find_highly_correlated_pairs(train_data)

[]


,Feature 1,Feature 2,Correlation


In [ ]:
categorical_cols = ['breast_cancer_diagnosis_code','breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code','patient_zip3', 'payer_type', 'patient_race', 'patient_state', 'Division', 'Region']

Cramers rule to find out how the categorical variables are correlated

In [ ]:
import scipy.stats as ss

def cramers_v(x, y):
  confusion_matrix = pd.crosstab(x, y)
  chi2 = ss.chi2_contingency(confusion_matrix)[0]
  n = confusion_matrix.sum().sum()
  phi2 = chi2 / n
  r, k = confusion_matrix.shape
  phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
  rcorr = r - ((r-1)**2)/(n-1)
  kcorr = k - ((k-1)**2)/(n-1)
  return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

def calculate_cramers_v_matrix(dataframe, categorical_cols):
  correlations_matrix = pd.DataFrame(index=categorical_cols, columns=categorical_cols)

  for col1 in categorical_cols:
    for col2 in categorical_cols:
      correlations_matrix.loc[col1, col2] = cramers_v(dataframe[col1], dataframe[col2])

  styled_correlations = correlations_matrix.style.background_gradient(cmap='coolwarm')\
            .format(precision=3)\
            .set_properties(**{'font-size': '9pt', 'font-family': 'Calibri'})\
            .set_table_styles([dict(selector='th', props=[('font-size', '10pt')])])\
            .set_caption("Cramér's V Correlations Matrix")

  return styled_correlations

cramers_v_matrix = calculate_cramers_v_matrix(train_data, categorical_cols)
print(display(cramers_v_matrix))


,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,patient_zip3,payer_type,patient_race,patient_state,Division,Region
breast_cancer_diagnosis_code,1.000,1.000,0.043,0.000,0.078,0.045,0.047,0.038,0.067
breast_cancer_diagnosis_desc,1.000,1.000,0.043,0.000,0.078,0.045,0.047,0.038,0.067
metastatic_cancer_diagnosis_code,0.043,0.043,1.000,0.049,0.077,0.041,0.000,0.037,0.055
patient_zip3,0.000,0.000,0.049,1.000,0.343,0.268,0.908,0.888,0.952
payer_type,0.078,0.078,0.077,0.343,1.000,0.214,0.308,0.186,0.160
patient_race,0.045,0.045,0.041,0.268,0.214,1.000,0.225,0.178,0.169
patient_state,0.047,0.047,0.000,0.908,0.308,0.225,1.000,0.996,0.994
Division,0.038,0.038,0.037,0.888,0.186,0.178,0.996,1.000,0.996
Region,0.067,0.067,0.055,0.952,0.160,0.169,0.994,0.996,1.000


None


Inference:

1. The fields patient_state, Division, Region, patient_zip3 are very much related to each other, which is expected since they are all geographical classifiers.

2. The breast_cancer_diagnosis_code and breast_cancer_diagnosis_desc are effectively duplicates of each other, carrying the same information. However, neither of these appears to be significantly related to the metastatic_cancer_diagnosis_code, suggesting that the metastatic cancer diagnosis is independent of the initial breast cancer diagnosis code and description.
